In [1]:
# Import my code
from tensorfactorization.utils import (defactorizing_CP, create_initial_data, random_cp_with_noise)
from tensorfactorization.multiplicative import (tensor_factorization_cp_multiplicative, tensor_factorization_cp_multiplicative_poisson)
from tensorfactorization.poisson import (BacktrackingWarning, tensor_factorization_cp_poisson, tensor_factorization_cp_poisson_fixed_step_size)

from toolkit.constants import (
    folder, data_folder, 
    error_label, iteration_label, tensor_dimension_label, time_label, 
    xscale_convergence_data, xscale_convergence, yscale_convergence
)

#%matplotlib widget
import tensorly as tl
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle # use pickle to save results to disk
import warnings

from copy import deepcopy

# Create Data

In [ ]:
# Trying do plot many stuff things, you know...
# TODO make this use GPU! For some reason this is super slow on GPU even though it should be very fast?
#tl.set_backend('pytorch')
#context = {'dtype': tl.float32,
#           'device': 'cuda'}
tl.set_backend('numpy')
context = {}

dimensions = []
# add large tensors
for _ in range(50):
    ndim = random.randint(3, 5)
    dimension = []

    max_dimension = 300*300*2
    # TODO make tensor generation smarter so that we dont generate massive ones!
    for n in range(ndim):
        next_dimension = random.randint(3, max( min( int(max_dimension / 3**(ndim-n -1)), 200) , 3 ) )
        dimension.append(next_dimension)
        max_dimension /= next_dimension
        
    dimensions.append(dimension)

# add small tensors
for _ in range(100):
    ndim = random.randint(3, 5)
    dimension = []

    for _ in range(ndim):
        next_dimension = random.randint(3, 12 )
        dimension.append(next_dimension)
        
    dimensions.append(dimension)

# let us also test different sigma values
alpha = 0.5
beta = 0.9
sigmas = [0.5, 0.1, 0.001]

points_for_sigma = {}


for sigma in sigmas:
    print(" \n \n Sigma = " + str(sigma))
    all_points = [] # list containing all points
    
    # big tensors only for now
    for dimension in dimensions:
        print("Dimension of new tensor: " + str(dimension))
        
        F = random.randint(2, 5) # get random order between 2 and 5
        max_iter = random.randint(6, 10) # get a random iteration between 5 and 10 
        scaling = random.uniform(0.1, 100.0) # get a random norm for our tensor
        
        data_of_tensor = []
        
        tensor = random_cp_with_noise(dimension, F, noise_scaling=0.0, context=context) # make it have no noise
        tensor = tensor * scaling
        norm_of_tensor = tl.norm(tensor)

        
        print("doing " + str(max_iter) + " iteration steps, with scaling: " +str(scaling)+" and norm: " + str(norm_of_tensor))

        # creating initial data
        initial_A_ns = create_initial_data(tensor, F)
        #norm_approx = tl.norm( defactorizing_CP(initial_A_ns, tensor.shape) )
        #scaling = (norm_of_tensor / norm_approx) ** (1.0/ tensor.ndim)
        #for n in range(len(initial_A_ns)):
        #    initial_A_ns[n] = initial_A_ns[n] * scaling
        backtracking_failed = False
        with warnings.catch_warnings(record=True) as w:
            # ignore runtimewarnings as they come from stuff that doesnt matter
            warnings.simplefilter("ignore", RuntimeWarning)
            # userwarnings are my warning that backtracking failed
            warnings.simplefilter("always", BacktrackingWarning)
            
            A_ns, _, _, step_size_modfiers = tensor_factorization_cp_poisson(tensor, F, max_iter=max_iter, detailed=True, verbose=False, sigma=sigma, beta=beta, initial_A_ns=initial_A_ns)
            print("done with factorization")
            
            if w:
                print("Warnings were issued during the function call, skipping tensor")
                if issubclass(warning.category, BacktrackingWarning):
                    print(f"Caught UserWarning: {warning.message}")
                    special_warning_issued = True
                else:
                    # Optionally store or print other captured warnings if you want to see what was ignored
                    print(f"Caught other warning (ignored): {warning.message} ({warning.category.__name__})")

        if backtracking_failed:
            continue
            
        for k in range(2): # always use the 2 last iterations to get bit more data points for free
            for n in range(tensor.ndim):
                khatr_rao_product = tl.tenalg.khatri_rao(A_ns, skip_matrix=n)
                data_of_tensor.append( {
                    "contraction" : khatr_rao_product.shape[0], 
                    "norm" : norm_of_tensor, 
                    "max" : tl.max(tensor),
                    "min" : tl.min(tensor),
                    "mean" : tl.mean(tensor),
                    "step_size" : alpha*math.pow(beta, step_size_modfiers[n][-k]) 
                } )

        # TODO: sort the points by contraction lenght so that plot looks better
        data_of_tensor = sorted(data_of_tensor, key=lambda x : x["contraction"])
        all_points.extend(data_of_tensor) # add the points from this tensor to the list of all points
        contraction_lenghts = [e["contraction"] for e in data_of_tensor]
        step_sizes = [e["step_size"] for e in data_of_tensor]
        plt.plot(contraction_lenghts, step_sizes)
    
    plt.xscale('log')
    plt.yscale('log')
    plt.title("lines Sigma = " + str(sigma))
    plt.show()
    
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # this time do the log ourselfs
    contraction_lenghts = [math.log(e["contraction"]) for e in all_points]
    norms = [e["norm"] for e in all_points]
    step_sizes = [math.log(e["step_size"]) for e in all_points]
    
    ax.scatter(contraction_lenghts, norms, step_sizes)
    #ax.set_xscale('log')
    #ax.set_yscale('log')

    ax.set_xlabel('contraction length')
    ax.set_ylabel('norm of tensor')
    ax.set_zlabel('step size')
    ax.set_title("Sigma = " + str(sigma))
    plt.show()

    points_for_sigma[sigma] = deepcopy(all_points)

try:
    pickle.dump( points_for_sigma, open(data_folder+'optimal_stepsize.pickle', 'wb') )
except Exception as e: print(e)

# Plot Data

In [ ]:
def fit_plane(points):
    """
    Fits the best 2D polynomial (plane) of the form z = ax + by + c to a set of 3D points.
    
    Args:
    points: A list or numpy array of 3D points, where each point is represented as [x, y, z].
    
    Returns:
    A tuple (a, b, c) representing the coefficients of the fitted plane.
    Returns None if there are fewer than 3 points.
    """
    points = np.array(points)
    if points.shape[0] < 3:
        print("Error: Need at least 3 points to fit a plane.")
        return None
    
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    
    # Construct the matrix A and vector b for the least squares fit
    # The equation is z = a*x + b*y + c, which can be written as:
    # [x1 y1 1] [a]   [z1]
    # [x2 y2 1] [b] = [z2]
    # ...       [c]   [...]
    A = np.vstack([x, y, np.ones(len(x))]).T
    b = z
    
    # Solve the least squares problem using numpy.linalg.lstsq
    coefficients, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    print("residual was " + str(residuals))
    
    # The coefficients are in the order [a, b, c]
    a, b, c = coefficients
    
    return a, b, c

In [ ]:
file = open(data_folder+"optimal_stepsize.pickle",'rb')
points_for_sigma = pickle.load(file)
file.close()

# iterate over all sigmas and print and plot the data
for sigma, all_points in points_for_sigma.items():
    print(" \n \n Sigma = " + str(sigma))
    
    contraction_lenghts = [math.log(e["contraction"]) for e in all_points]
    scaling_types = {
        "norm" : [e["norm"] for e in all_points],
        "max" : [e["max"] for e in all_points],
        "min" : [e["min"] for e in all_points],
        "mean" : [e["mean"] for e in all_points],
    }
    step_sizes = [math.log(e["step_size"]) for e in all_points]

    for name, data in scaling_types.items():
        print("Now testing " + name)
        
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.scatter(contraction_lenghts, data, step_sizes)
        #ax.set_xscale('log')
        #ax.set_yscale('log')
    
        ax.set_xlabel('contraction length')
        ax.set_ylabel(name+' of tensor')
        ax.set_zlabel('step size')
        ax.set_title(name+" and sigma = " + str(sigma))
        #plt.show()
    
        # let us now find the best 1d line through these points in loglog
        points = [(math.log(p["contraction"]), p[name], math.log(p["step_size"])) for p in all_points]
        coefficients = fit_plane(points)
        print(coefficients)
        # recalculate to exponential formula
        print("so our formula is: step size = "+str(math.pow(10, coefficients[2]))+" * length^("+str(coefficients[0])+") * e^("+str(coefficients[1])+" "+name+")")
        #p = np.poly1d(z)
        #xp = np.linspace(min(x)-1, max(x)+1, 100)
        
        #_ = plt.plot(x, y, '.', xp, p(xp), '-')
        #plt.title("Sigma = " + str(sigma))
        a, b, c = coefficients
        xp = np.linspace(min(contraction_lenghts)-1, max(contraction_lenghts)+1, 100)
        yp = np.linspace(min(data)-1, max(data)+1, 100)
        X, Y = np.meshgrid(xp, yp)
        Z = a * X + b * Y + c
        ax.plot_surface(X, Y, Z, color='red', alpha=0.5, label='Fitted Plane')
        plt.show()